# Storage Access API Demo Analysis

This jupyter notebook can be used to extract the results of the SAA experiment from the database and analyze it.

In [ ]:
# Automatically reload all modules that are changed before executing code
%load_ext autoreload
%autoreload 2

---

## Imports and Helper Functions

In [ ]:
from database import Task, URL
from IPython.display import display, HTML
from modules.storageaccessapi import Document, DocumentInclusion, Script, ScriptInclusion
from peewee import fn
import matplotlib.pyplot as plt
import pandas as pd

---

## Browser Differences

### SAA Scripts found

In [ ]:
scripts = []
labels = ["chromium", "firefox", "webkit"]
tables = []

for browser in labels:
    query = (
            Script
            .select(
                Script.url, Script.sha1, Script.id.alias("script_id")
            )
            .join(ScriptInclusion, on=(ScriptInclusion.script == Script.id))
            .where((Script.saa == True) & (ScriptInclusion.browser == browser))
            .distinct(Script.url)
    )
    tables.append(pd.DataFrame(list(query.dicts()))["url"])
    scripts.append(len(pd.DataFrame(list(query.dicts()))))

print(scripts)
plt.figure(figsize=(8, 5))
plt.bar(labels, scripts, color=['blue', 'green', 'red'])
plt.xlabel('Browser')
plt.ylabel('SAA Scripts')
plt.title('Number of Scripts using SAA')
plt.show()

In [ ]:
intersection1 = pd.merge(tables[0], tables[1], on='url', how='inner')
intersection2 = pd.merge(tables[0], tables[2], on='url', how='inner')
intersection3 = pd.merge(tables[1], tables[2], on='url', how='inner')
percentage1 = (len(intersection1) / len(tables[0])) * 100
percentage2 = (len(intersection2) / len(tables[0])) * 100
percentage3 = (len(intersection3) / len(tables[1])) * 100
print("Intersection Chrome / Firefox:", percentage1)
print("Intersection Chrome / Webkit:", percentage2)
print("Intersection Firefox / Webkit:", percentage3)

set1 = set(tables[0])
set2 = set(tables[1])
unique_df1 = set1 - set2
unique_df2 = set2 - set1
print(len(unique_df1))
print(len(unique_df2))

### SAA Documents found

In [ ]:
documents = []
labels = ["chromium", "firefox", "webkit"]
tables = []

for browser in labels:
    query = (
            Document
            .select(
                Document.url, Document.sha1, Document.id.alias("document_id")
            )
            .join(DocumentInclusion, on=(DocumentInclusion.document == Document.id))
            .where((Document.saa == True) & (DocumentInclusion.browser == browser))
            .distinct(Document.url)
    )
    tables.append(pd.DataFrame(list(query.dicts()))["url"])
    documents.append(len(pd.DataFrame(list(query.dicts()))))

print(documents)
plt.figure(figsize=(8, 5))
plt.bar(labels, documents, color=['blue', 'green', 'red'])
plt.xlabel('Browser')
plt.ylabel('SAA Documents')
plt.title('Number of Documents using SAA')
plt.show()

In [ ]:
intersection1 = pd.merge(tables[0], tables[1], on='url', how='inner')
intersection2 = pd.merge(tables[0], tables[2], on='url', how='inner')
intersection3 = pd.merge(tables[1], tables[2], on='url', how='inner')
percentage1 = (len(intersection1) / len(tables[0])) * 100
percentage2 = (len(intersection2) / len(tables[0])) * 100
percentage3 = (len(intersection3) / len(tables[1])) * 100
print("Intersection Chrome / Firefox:", percentage1)
print("Intersection Chrome / Webkit:", percentage2)
print("Intersection Firefox / Webkit:", percentage3)

---

## Task Analysis

In [ ]:
# Select data from db
tasks = Task.select().where(Task.job == "storageaccessapi")
urls = URL.select().where(URL.job == "storageaccessapi")

# Transform to dataframes
tasks = pd.DataFrame(list(tasks.dicts()))
urls = pd.DataFrame(list(urls.dicts()))

# Show tasks table
display(tasks)

### Task Errors

In [ ]:
errors = Task.select().where(Task.error != None)
errors = pd.DataFrame(list(errors.dicts()))
display(errors)

---

## All Scripts and Documents using SAA

In [ ]:
query_scripts = (
        Script
        .select(
            Script.url, Script.id.alias("script_id")
        )
        .where(Script.saa == True)
)

query_documents = (
        Document
        .select(
            Document.url, Document.id.alias("document_id")
        )
        .where(Document.saa == True)
)

saa_scripts = pd.DataFrame(list(query_scripts.dicts()))
saa_documents = pd.DataFrame(list(query_documents.dicts()))
display(HTML("<h3>Scripts:</h3>"))
display(saa_scripts)
display(HTML("<h3>Documents:</h3>"))
display(saa_documents)

---

## All Script- and DocumentInclusion using SAA


In [ ]:
query_scripts = (
        ScriptInclusion
        .select(
            ScriptInclusion, Script.url
        )
        .join(Script, on=(ScriptInclusion.script == Script.id))
        .where(Script.saa == True)
)

query_documents = (
        DocumentInclusion
        .select(
            DocumentInclusion, Document.url
        )
        .join(Document, on=(DocumentInclusion.document == Document.id))
        .where(Document.saa == True)
)

saa_scripts = pd.DataFrame(list(query_scripts.dicts()))
saa_documents = pd.DataFrame(list(query_documents.dicts()))
display(HTML("<h3>Scripts:</h3>"))
display(saa_scripts)
display(HTML("<h3>Documents:</h3>"))
display(saa_documents)

print(list(dict.fromkeys([x["url"] for x in list(query_scripts.dicts()) if "captcha" not in x["url"] and "gstatic" not in x["url"]])))

---

## Analysis of Sites using SAA

### Sites using SAA

In [ ]:
# Only sites using SAA are stored in the DB
# We only need to query the DocumentInclusion with distinct site column
query = (
    DocumentInclusion
    .select(DocumentInclusion.site)
    .distinct(DocumentInclusion.site)
)

# Execute the query and return the results
top_level_sites = pd.DataFrame(list(query.dicts()))
display(top_level_sites)

---

In [ ]:
site = "dominos.com"

### Top-level URLs of Site embedding SAA Content

In [ ]:
# Get all Documents that have top-level DocumentInclusions
query = (
        DocumentInclusion
        .select(
            DocumentInclusion, Document
        )
        .join(Document, on=(DocumentInclusion.document == Document.id))
        .where((DocumentInclusion.site == site) & (DocumentInclusion.parent.is_null(True)))
)

top_level_documents = pd.DataFrame(list(query.dicts()))
display(top_level_documents)

### Document- and ScriptInclusions of Top-level Site using SAA

In [ ]:
# Get all Documents using SAA embedded on the top-level site
TopLevelDocument = Document.alias('TopLevelDocument')
query = (
        DocumentInclusion
        .select(
            Document, TopLevelDocument.url.alias("top_level_url")
        )
        .join(Document, on=(DocumentInclusion.document == Document.id))
        .join(TopLevelDocument, on=(DocumentInclusion.top_level_site == TopLevelDocument.id))
        .where((DocumentInclusion.site == site) & (Document.saa))
)

documents = pd.DataFrame(list(query.dicts()))
display(documents)

In [ ]:
# Get all Scripts using SAA embedded on the top-level site
query = (
        ScriptInclusion
        .select(
            ScriptInclusion.id, ScriptInclusion.document_inclusion, Script.sha1, Script.sha1_url, Script.url.alias("script_url"), Script.saa, Document.url.alias("top_level_url")
        )
        .join(Script, on=(ScriptInclusion.script == Script.id))
        .join(Document, on=(ScriptInclusion.top_level_site == Document.id))
        .where((ScriptInclusion.site == site) & (Script.saa))
)

scripts = pd.DataFrame(list(query.dicts()))
display(scripts)

print(scripts["script_url"][0])

### Frame hierarchy of specific URL

In [ ]:
class Frame:

    def __init__(self, sha1, sha1_url, url, saa, parent=None):
        self.sha1 = sha1
        self.sha1_url = sha1_url
        self.url = url
        self.scripts = []
        self.saa = saa
        self.parent = parent
        self.children = []


class Scripting:

    def __init__(self, sha1, sha1_url, url, saa):
        self.sha1 = sha1
        self.sha1_url = sha1_url
        self.url = url
        self.saa = saa

In [ ]:
def get_tree(inclusion_id, frame:Frame=None):
    # Get document inclsuion from id
    docinc = DocumentInclusion.get(DocumentInclusion.id == inclusion_id)

    # If frame is None we need to set the top-level frame, otherwise append as child to the previous frame
    if frame is None:
        frame = Frame(sha1=docinc.document.sha1, sha1_url=docinc.document.sha1_url, url=docinc.document.url, saa=docinc.document.saa)
    else:
        new_frame = Frame(sha1=docinc.document.sha1, sha1_url=docinc.document.sha1_url, url=docinc.document.url, saa=docinc.document.saa, parent=frame)
        frame.children.append(new_frame)
        frame = new_frame

    # Add scripts
    for scriptinc in docinc.script_inclusions:
        frame.scripts.append(Scripting(sha1=scriptinc.script.sha1, sha1_url=scriptinc.script.sha1_url, url=scriptinc.script.url, saa=scriptinc.script.saa))

    # Recursively run the function on all frame children
    for child in docinc.children:
        get_tree(child.id, frame)

    return frame

In [ ]:
print(get_tree(2274).children[2].url)

---

### Get Content of Script or Document

In [ ]:
def get_content(sha1, sha1_url):
    with open(f"/pycrawler/file_storage/{'/'.join(list(sha1))}/{sha1_url}", "r") as fd:
        content = fd.read()
        return content

In [ ]:
print(get_content("60b37296f3229b08038fe7af4b4a4d339f842933", "aa1cc3a44ef7c8ee22ef57012cc63ae3a28203c1"))